In [1]:
#random crashes plus can move into itself
import pygame
import random 
import numpy as np

# Call this function so the Pygame library can initialize itself
pygame.init()
 
# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Set the width and height of each snake segment
segment_width = 15
segment_height = 15

# Margin between each segment
segment_margin = 3
 
# Set initial speed
x1_change = segment_width + segment_margin
y1_change = 0


screen_w = int(input('What do you want the screen width to be?'))
screen_w = round(screen_w/(segment_width+segment_margin)) * (segment_width+segment_margin)
screen_h = int(input('What do you want the screen height to be?'))
screen_h = round(screen_h/(segment_height+segment_margin)) * (segment_height+segment_margin)
print(screen_w,screen_h)

grid_height = 0
grid_height = int(screen_h/(segment_height+segment_margin))

grid_width = 0
grid_width = int(screen_w/(segment_width+segment_margin))
screen = pygame.display.set_mode([screen_w, screen_h]) #set the display   

#define class Apple
class Apple(pygame.sprite.Sprite):
    def __init__(self,x,y):
        # Call the parent's constructor
        super().__init__()        
        # Set height, width
        self.image = pygame.Surface([segment_width, segment_height])
        self.image.fill((255,0,0))
        # Make our top-left corner the passed-in location.
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

#define call Segment
class Segment(pygame.sprite.Sprite):
    """ Class to represent one segment of the snake. """
    # -- Methods
    # Constructor function
    def __init__(self, x, y):
        # Call the parent's constructor
        super().__init__()
 
        # Set height, width
        self.image = pygame.Surface([segment_width, segment_height])
        self.image.fill(WHITE)
        # Make our top-left corner the passed-in location.
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

determined = False # for apple later
eaten = True # for apple later

# Set the title of the window
pygame.display.set_caption('Snake Example')
 
allspriteslist1 = pygame.sprite.Group() #create group
 
# Create an initial snake
#create initial list
snake_segments1 = []
snake_segments_pos1 = []

for i in range(10):
    x1 = 146 - (segment_width + segment_margin) * i
    y1 = 20
    segment = Segment(x1, y1)
    snake_segments1.append(segment)
    snake_segments_pos1.append((segment.rect.x,segment.rect.y))
    allspriteslist1.add(segment)


clock = pygame.time.Clock()
done = False
#running 
while not done:
    #setting up grid starting of apple
    grid = np.arange(0,grid_width*grid_height).reshape(grid_width,grid_height)   
    #picking location of apple

    if eaten == True:
        determined = False # reset it for next apple
        #setting up apple coords
        apple_coordx = 0
        apple_coordy = 0
        
        while determined == False:
            pick = np.where(grid == random.randint(0,(len(grid))))
            
            if pick != 0:
                pick_index = pick[0]*pick[1]-pick[1]
                apple_coordx = pick[0] * (segment_width+segment_margin) + 2 #margin of error change to segment_margin-1 ?
                apple_coordy = pick[1] * (segment_width+segment_margin) + 2
                if tuple((apple_coordx,apple_coordy)) not in snake_segments_pos1:
                    determined = True
                    #print(apple_coordx,apple_coordy)
                else:
                    np.delete(grid,[pick[0],pick[1]])
                     
                    #np.insert(grid,[pick[0],pick[1]],int(0))
                    #print(pick)
                    #print(grid)

        #removing old apple
        if len(allspriteslist1) > 11:
            allspriteslist1.remove(apple)
        #giving apple coords
        apple = Apple(apple_coordx,apple_coordy) 
        allspriteslist1.add(apple)

        eaten = False # set back to default ending of apple
        
        #detecting input
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
            
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                done = True
                
            if event.key == pygame.K_LEFT and x1_change != segment_width + segment_margin:
                x1_change = (segment_width + segment_margin) * -1
                y1_change = 0
                
            if event.key == pygame.K_RIGHT and x1_change != (segment_width + segment_margin) * -1:
                x1_change = (segment_width + segment_margin)
                y1_change = 0
                
            if event.key == pygame.K_UP and y1_change != segment_width + segment_margin:
                x1_change = 0
                y1_change = (segment_height + segment_margin) * -1
                
            if event.key == pygame.K_DOWN and y1_change != (segment_width + segment_margin) * -1:
                x1_change = 0
                y1_change = (segment_height + segment_margin)
                
            if event.key == pygame.K_a:
                print(snake_segments_pos1)
                print(apple_coordx,apple_coordy)
                print(allspriteslist1)
                
    #check if apple got
    if snake_segments_pos1[0] == tuple((apple_coordx,apple_coordy)):
        eaten = True
    if eaten == True:
        snake_segments1.append(Segment(x, y))
        snake_segments_pos1.append((x,y))
    
    #changing pos list
    for a in (range(len(snake_segments1)-1)):
        snake_segments_pos1[-a-1] = snake_segments_pos1[-a-2]
        
    #deleting old 
    old_segment = snake_segments1.pop()
    allspriteslist1.remove(old_segment)
    
    #changing head
    x = snake_segments1[0].rect.x + x1_change
    y = snake_segments1[0].rect.y + y1_change
    snake_segments_pos1[0] = (x,y)
    segment = Segment(x, y)
        
    #self-collision
    if snake_segments_pos1[0] in snake_segments_pos1[1:]:
        done = True
        
    #wall-collision
    if snake_segments_pos1[0][0] > screen_w - 1:
        done = True
    if snake_segments_pos1[0][0] < 0: 
        done = True
    if snake_segments_pos1[0][1] < 0:
        done = True
    if snake_segments_pos1[0][1] > screen_h - 1:
        done = True
        
    # Insert new segment into the list
    snake_segments1.insert(-1*len(allspriteslist1), segment)
    allspriteslist1.add(segment)
    
    # Clear screen
    screen.fill(BLACK)

    for x in range(grid_width):
        pygame.draw.line(screen,(25,25,25),(x*(segment_width+segment_margin),0),(x*(segment_width+segment_margin),screen_h),3)
    for x in range(grid_height):    
        pygame.draw.line(screen,(25,25,25),(0,x*(segment_height+segment_margin)),(screen_w,x*(segment_height+segment_margin)),3) 
    allspriteslist1.draw(screen)

    # Flip screen
    pygame.display.flip()
  
    # Pause
    clock.tick(5)
#end of game
screen.fill(BLACK)

print(f'You lose! You got {(len(allspriteslist1)-11)} apples.')

pygame.quit()

pygame 2.0.0.dev6 (SDL 2.0.10, python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
What do you want the screen width to be?400
What do you want the screen height to be?400
396 396
You lose! You got 2 apples.
